# HW6 FX Carry (GBP Funding) — Single Source of Truth

In [ ]:
# RUN MANIFEST + environment
from pathlib import Path
from datetime import datetime, timezone
import json, subprocess, platform, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE = Path('.').resolve()
OUTPUT_DIR = BASE / 'outputs'
FIG_DIR = OUTPUT_DIR / 'figures'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

manifest = {
    'timestamp_utc': datetime.now(timezone.utc).isoformat(),
    'python': platform.python_version(),
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'base_dir': str(BASE),
    'files_chosen': {},
    'coverage': {},
    'weekly_obs': None,
    'n_currencies': None,
}

try:
    git_commit = subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD'], text=True).strip()
except Exception:
    git_commit = 'unknown'
manifest['git_commit'] = git_commit
print('timestamp_utc:', manifest['timestamp_utc'])
print('git_commit:', git_commit)
print('python/pandas/numpy:', manifest['python'], manifest['pandas'], manifest['numpy'])

In [ ]:
# Deterministic discovery utilities

def inventory_tree(root: Path, limit: int = 30):
    print(f'Inventory: {root}')
    if not root.exists():
        print('  <missing>')
        return []
    files = sorted([p for p in root.glob('*') if p.is_file()], key=lambda x: x.name)
    for p in files[:limit]:
        st = p.stat()
        print(f'  {p.name:55s} size={st.st_size:8d} mtime={pd.Timestamp(st.st_mtime, unit="s")}')
    return files


def find_one(root: Path, patterns, manifest_key: str):
    if isinstance(patterns, str):
        patterns = [patterns]
    candidates = []
    for pat in patterns:
        candidates.extend(list(root.glob(pat)))
    candidates = sorted(set(candidates), key=lambda p: (p.stat().st_mtime, p.name), reverse=True)
    print(f'find_one root={root} patterns={patterns}')
    print('  candidates:', [c.name for c in candidates])
    if not candidates:
        inventory_tree(root)
        raise FileNotFoundError(f'No file matched patterns={patterns} in {root}')
    chosen = candidates[0]
    print('  chosen:', chosen)
    manifest['files_chosen'][manifest_key] = str(chosen)
    return chosen


def read_any(path: Path):
    suf = path.suffix.lower()
    if suf == '.parquet':
        return pd.read_parquet(path)
    if suf == '.csv':
        return pd.read_csv(path)
    if suf in ('.xlsx', '.xls'):
        return pd.read_excel(path)
    raise ValueError(f'Unsupported file type: {path}')

inventory_tree(BASE/'data_clean')
inventory_tree(BASE/'../../Data')
inventory_tree(BASE/'data')

In [ ]:
# Data loading
EM = ['BRL','NGN','PKR','TRY','ZAR']

# OIS overnight (accrual)
ois_fp = find_one(BASE/'data_clean', ['*IUDSOIA*.parquet','*IUDSOIA*.csv'], 'ois_overnight')
ois_df = read_any(ois_fp)
if isinstance(ois_df.index, pd.DatetimeIndex):
    ois = pd.Series(pd.to_numeric(ois_df.iloc[:,0], errors='coerce').values,
                    index=pd.to_datetime(ois_df.index, errors='coerce'), name='ois_on')
else:
    dcol = next((c for c in ois_df.columns if str(c).upper()=='DATE'), ois_df.columns[0])
    vcol = next((c for c in ois_df.columns if 'IUDSOIA' in str(c).upper()), [c for c in ois_df.columns if c!=dcol][0])
    ois = pd.Series(pd.to_numeric(ois_df[vcol], errors='coerce').values,
                    index=pd.to_datetime(ois_df[dcol], errors='coerce'), name='ois_on')
ois = ois.dropna().sort_index()
ois.index = ois.index.tz_localize(None)
if ois.median() > 1:
    ois = ois / 100.0

# FX USD per CCY
fx_fp = find_one(BASE/'data_clean', ['*usd_per_ccy*wide*.parquet','*edi*cur*fx_long*.parquet'], 'fx')
fx_raw = read_any(fx_fp)
if {'date','ccy','value'}.issubset(set(getattr(fx_raw,'columns',[]))):
    fx = fx_raw.pivot(index='date', columns='ccy', values='value')
else:
    fx = fx_raw.copy()
if not isinstance(fx.index, pd.DatetimeIndex):
    fx.index = pd.to_datetime(fx.index, errors='coerce')
fx.index = fx.index.tz_localize(None)
fx.columns = [str(c).upper().strip() for c in fx.columns]
fx = fx.apply(pd.to_numeric, errors='coerce').sort_index()
for c in sorted(set(EM+['GBP'])):
    if c not in fx.columns:
        fx[c] = np.nan
fx = fx[sorted(set(EM+['GBP']))]

gbp = fx['GBP'].dropna()
if gbp.between(0.3,5.0).mean() < 0.95:
    fx = 1.0 / fx
    print('FX auto-inversion applied (GBP range sanity).')
assert fx['GBP'].dropna().between(0.3,5.0).mean() > 0.95, 'GBP quote sanity failed after inversion logic.'
print('FX >20% daily move counts:')
print((fx.pct_change().abs()>0.2).sum())

# EM curves from ../../Data preferred, fallback ./data
em_candidates = sorted(list((BASE/'../../Data').glob('*Emerging*Mkt*YC*.csv')) + list((BASE/'data').glob('*Emerging*Mkt*YC*.csv')),
                       key=lambda p:(p.stat().st_mtime,p.name), reverse=True)
if not em_candidates:
    raise FileNotFoundError('No EM curve csv found in ../../Data or ./data')
print('EM candidates:', [x.name for x in em_candidates])
manifest['files_chosen']['em_curve_files'] = [str(x) for x in em_candidates]

def parse_em_csv(fp):
    raw = pd.read_csv(fp)
    out=[]
    cols=list(raw.columns)
    for i in range(0,len(cols),2):
        c0 = cols[i]
        c1 = cols[i+1] if i+1 < len(cols) else None
        if c1 is None:
            continue
        m = re.search(r'GT([A-Z]{3})(\d+)(Y|YR)', str(c0).upper())
        if not m:
            continue
        tmp = pd.DataFrame({
            'date': pd.to_datetime(raw[c0], errors='coerce'),
            'ccy': m.group(1),
            'tenor': float(m.group(2)),
            'rate': pd.to_numeric(raw[c1], errors='coerce')
        }).dropna(subset=['date','rate'])
        out.append(tmp)
    if not out:
        raise RuntimeError(f'No parsable columns in {fp}')
    return pd.concat(out, ignore_index=True)

curves = pd.concat([parse_em_csv(f) for f in em_candidates], ignore_index=True)
curves['date'] = pd.to_datetime(curves['date']).dt.tz_localize(None)
if curves['rate'].median() > 1:
    curves['rate'] = curves['rate']/100.0
curves = curves.groupby(['date','ccy','tenor'], as_index=False)['rate'].mean()

# Funding 5Y construction (fail loudly if impossible)
boe_raw_fp = find_one(BASE/'data_clean', ['*boe*ois*daily*raw*.parquet','*ois*daily*raw*.parquet'], 'gbp_curve_raw')
boe_raw = pd.read_parquet(boe_raw_fp)

required_cols = {'__source_file','__sheet','UK OIS spot curve'}
if not required_cols.issubset(boe_raw.columns):
    print('Columns seen in boe raw:', boe_raw.columns.tolist())
    raise RuntimeError('Cannot construct GBP 5Y: required columns missing in boe raw dataset')

spot_rows = boe_raw[boe_raw['__sheet'].astype(str).str.contains('spot curve', case=False, na=False)].copy()
if spot_rows.empty:
    raise RuntimeError('Cannot construct GBP 5Y: no spot curve rows available')

parts=[]
for src, g in spot_rows.groupby('__source_file'):
    vals = pd.to_numeric(g['UK OIS spot curve'], errors='coerce').dropna().reset_index(drop=True)
    vals = vals[(vals>-5) & (vals<30)]
    # remove tenor marker rows if present
    if len(vals)>2 and abs(vals.iloc[0]-1.0)<1e-10 and abs(vals.iloc[1]-1/12)<1e-10:
        vals = vals.iloc[2:].reset_index(drop=True)
    if len(vals)<100:
        continue
    years = [int(x) for x in re.findall(r'(20\d{2})', str(src))]
    if years:
        y0,y1 = min(years), max(years)
        idx = pd.DatetimeIndex([d for d in ois.index if y0<=d.year<=y1])
    else:
        idx = ois.index
    if len(idx) < len(vals):
        idx = ois.index[-len(vals):]
    else:
        idx = idx[:len(vals)]
    parts.append(pd.Series(vals.values, index=idx))

if not parts:
    print('data_clean inventory:'); inventory_tree(BASE/'data_clean')
    print('patterns attempted: *boe*ois*daily*raw*.parquet, *ois*daily*raw*.parquet')
    print('boe raw columns:', boe_raw.columns.tolist())
    raise RuntimeError('Cannot construct GBP 5Y funding series from local data.')

gbp5 = pd.concat(parts).sort_index()
gbp5 = gbp5[~gbp5.index.duplicated(keep='last')].dropna()
# BoE spot values appear in percent units; convert to decimal robustly
a = gbp5.median()
if a > 0.2:
    gbp5 = gbp5 / 100.0
gbp5.name='s5_fund'
manifest['files_chosen']['gbp5y_method'] = 'BoE OIS spot-curve raw parsed into GBP 5Y-like funding proxy from local archive'

In [ ]:
# Canonical weekly alignment: WED with asof forward within +2 days

def align_forward_2d(obj, targets):
    idx = obj.index
    if isinstance(obj, pd.Series):
        vals=[]
        for d in targets:
            cand=[d+pd.Timedelta(days=k) for k in [0,1,2]]
            c=next((x for x in cand if x in idx), None)
            vals.append(np.nan if c is None else obj.loc[c])
        return pd.Series(vals, index=targets, name=obj.name)
    rows=[]
    for d in targets:
        cand=[d+pd.Timedelta(days=k) for k in [0,1,2]]
        c=next((x for x in cand if x in idx), None)
        if c is None:
            rows.append(pd.Series(np.nan, index=obj.columns))
        else:
            rows.append(pd.Series(obj.loc[c].values, index=obj.columns))
    return pd.DataFrame(rows, index=targets)

curve_wide = curves.pivot_table(index=['date','ccy'], columns='tenor', values='rate', aggfunc='mean').sort_index()

# initial broad window
t_start = max(ois.index.min(), gbp5.index.min(), fx.index.min(), curves['date'].min())
t_end = min(ois.index.max(), gbp5.index.max(), fx.index.max(), curves['date'].max())
all_targets = pd.date_range(t_start, t_end, freq='W-WED')

def has_match(idx, d):
    return any((d+pd.Timedelta(days=k)) in idx for k in [0,1,2])

# keep only weeks where all required datasets have forward+2d matches
valid_targets=[]
curve_idx = {c: curve_wide.xs(c, level='ccy').index for c in EM}
for d in all_targets:
    ok = has_match(ois.index,d) and has_match(gbp5.index,d) and has_match(fx.index,d)
    if ok:
        for c in EM:
            ok = ok and has_match(curve_idx[c], d)
            if not ok:
                break
    if ok:
        valid_targets.append(d)
targets = pd.DatetimeIndex(valid_targets)

if len(targets)==0:
    raise RuntimeError('No valid weekly targets after forward+2d alignment constraints')

ois_w = align_forward_2d(ois, targets)
gbp5_w = align_forward_2d(gbp5, targets)
fx_w = align_forward_2d(fx, targets)
dates = targets
curve_ccy = {c: align_forward_2d(curve_wide.xs(c,level='ccy'), targets) for c in EM}

# Missingness checks
miss = {}
miss['ois_on'] = float(ois_w.isna().mean())
miss['gbp5_fund'] = float(gbp5_w.isna().mean())
for c in fx_w.columns:
    miss[f'fx_{c}'] = float(fx_w[c].isna().mean())
for c in EM:
    miss[f'curve_{c}'] = float(curve_ccy[c].isna().all(axis=1).mean())
miss_df = pd.DataFrame({'series':list(miss.keys()), 'missing_frac':list(miss.values())})
miss_df.to_csv(OUTPUT_DIR/'alignment_missingness.csv', index=False)
print(miss_df)

for c in EM:
    if miss[f'fx_{c}'] > 0.05:
        raise RuntimeError(f'FX missing >5% for {c}')
    if miss[f'curve_{c}'] > 0.05:
        raise RuntimeError(f'Curve missing >5% for {c}')
if miss['gbp5_fund'] > 0.05:
    raise RuntimeError('GBP funding 5Y missing >5%')

# fill small residual holes after guardrail
ois_w = ois_w.ffill()
gbp5_w = gbp5_w.ffill()
fx_w = fx_w.ffill()
for c in EM:
    curve_ccy[c] = curve_ccy[c].ffill()

manifest['coverage'] = {
    'start': str(dates.min().date()),
    'end': str(dates.max().date()),
    'weekly_obs': int(len(dates))
}
manifest['weekly_obs'] = int(len(dates))
manifest['n_currencies'] = int(len(EM))
print('Weekly targets kept:', len(dates), dates.min().date(), dates.max().date())


In [ ]:
# Curve pricing and MTM shift validation

def interp_rate(tenors, rates, t):
    x=np.array(tenors,dtype=float); y=np.array(rates,dtype=float)
    m=np.isfinite(x)&np.isfinite(y)
    x=x[m]; y=y[m]
    if len(x)==0: return np.nan
    o=np.argsort(x); x=x[o]; y=y[o]
    return float(np.interp(t, x, y, left=y[0], right=y[-1]))

def bootstrap_df_from_par(par_curve, freq=4, max_t=5.0):
    grid=np.arange(1/freq, max_t+1e-12, 1/freq)
    dfs={}
    for t in grid:
        s=interp_rate(par_curve.index.values, par_curve.values, t)
        c=s/freq
        prev=sum(c*dfs[pt] for pt in grid if pt<t)
        dfs[t]=max((1-prev)/(1+c),1e-12)
    return pd.Series(dfs)

def price_bond_shifted(coupon_rate, curve_row, times, freq=4):
    if len(times)==0:
        return 1.0
    z=bootstrap_df_from_par(curve_row, freq=freq, max_t=max(5.0,float(np.max(times))+0.25))
    df=np.interp(times, z.index.values, z.values, left=z.values[0], right=z.values[-1])
    cf=np.full(len(times), coupon_rate/freq)
    cf[-1]+=1.0
    return float(np.sum(cf*df))

times_entry = np.arange(0.25, 5.0+1e-12, 0.25)
dt = 1/52
times_exit_full = times_entry - dt
times_exit = times_exit_full[times_exit_full > 0]
print('entry first/last5:', times_entry[:5], times_entry[-5:])
print('exit_full first/last5:', times_exit_full[:5], times_exit_full[-5:])
assert np.allclose(times_exit_full, times_entry - dt)

# par sanity sample
rng=np.random.default_rng(7)
par_checks=[]
for c in EM:
    cdf=curve_ccy[c]
    valid_idx=np.where(~cdf.isna().all(axis=1))[0]
    if len(valid_idx)==0: continue
    picks=rng.choice(valid_idx, size=min(5,len(valid_idx)), replace=False)
    for j in picks:
        row=cdf.iloc[int(j)].dropna()
        s5=interp_rate(row.index.values,row.values,5.0)
        pv0=price_bond_shifted(s5,row,times_entry)
        par_checks.append({'ccy':c,'date':str(cdf.index[int(j)].date()),'s5':s5,'pv0':pv0,'abs_dev':abs(pv0-1)})
par_df=pd.DataFrame(par_checks)
print(par_df.head(10))
if not par_df.empty and (par_df['abs_dev']>0.02).any():
    print('Par sanity diagnostics failures:')
    print(par_df[par_df['abs_dev']>0.02].head(20))
    raise RuntimeError('Par sanity check failed (abs(PV0-1)>0.02)')

In [ ]:
# Strategy simulation
rows=[]
port_rows=[]

for i in range(len(dates)-1):
    t0, t1 = dates[i], dates[i+1]
    s5_fund = float(gbp5_w.loc[t0])
    ois_on = float(ois_w.loc[t0])
    borrow_rate = ois_on + 0.005

    active_rets=[]
    active_count=0
    spreads_this_week=[]
    for c in EM:
        c0 = curve_ccy[c].loc[t0].dropna()
        c1 = curve_ccy[c].loc[t1].dropna()
        if c0.empty or c1.empty:
            continue
        s5_lend = interp_rate(c0.index.values, c0.values, 5.0)
        spread = s5_lend - s5_fund
        trade = bool(np.isfinite(s5_lend) and np.isfinite(s5_fund) and spread >= 0.005)

        f0 = fx_w.at[t0,c]; f1 = fx_w.at[t1,c]
        g0 = fx_w.at[t0,'GBP']; g1 = fx_w.at[t1,'GBP']
        if not np.isfinite([f0,f1,g0,g1]).all():
            trade = False

        ret=np.nan; pnl=0.0
        if trade:
            units_ccy = 10_000_000 / f0
            pv1 = price_bond_shifted(s5_lend, c1, times_exit)
            lend_end_usd = units_ccy * pv1 * f1

            debt_units_gbp = 8_000_000 / g0
            debt_end_usd = debt_units_gbp * (1 + borrow_rate/52) * g1

            eq0 = 2_000_000
            eq1 = lend_end_usd - debt_end_usd
            pnl = eq1 - eq0
            ret = pnl / eq0
            # equity cannot lose more than 100% in a week
            ret = max(ret, -0.999999)
            active_rets.append(ret)
            active_count += 1

        rows.append({
            'date':t0,'next_date':t1,'ccy':c,'active':int(trade),
            's5_lend':s5_lend,'s5_fund':s5_fund,'spread':spread,
            'ois_on':ois_on,'borrow_rate':borrow_rate,'ret':ret,'pnl_usd':pnl
        })
        spreads_this_week.append(spread)

    port_ret = float(np.mean(active_rets)) if active_rets else 0.0
    port_rows.append({'date':t0,'port_ret':port_ret,'active_positions':active_count,'spread_mean':float(np.mean(spreads_this_week)) if spreads_this_week else np.nan})

res = pd.DataFrame(rows)
port = pd.DataFrame(port_rows).set_index('date').sort_index()
port['wealth'] = (1+port['port_ret']).cumprod()
port['drawdown'] = port['wealth']/port['wealth'].cummax()-1
assert float(port['drawdown'].min()) >= -1 - 1e-12

# If suspiciously always active, print spread diagnostics
avg_active = float(port['active_positions'].mean())
print('avg_active_positions_per_week:', avg_active)
if avg_active > 4.9:
    print('SUSPICIOUS activity. spread diagnostics by currency:')
    print(res.groupby('ccy')['spread'].describe())

In [ ]:
# Output tables with exact semantics

def sharpe_weekly(r):
    s=r.std(ddof=1)
    return (r.mean()/s) if s>0 else np.nan

def wealth_dd_from_returns(r):
    w=(1+r).cumprod()
    dd=w/w.cummax()-1
    return w,dd

# portfolio output
portfolio_out = port.reset_index()[['date','port_ret','wealth','drawdown','active_positions']]
portfolio_out.to_csv(OUTPUT_DIR/'portfolio_weekly_returns.csv', index=False)

# currency stats
stats=[]
for c in EM:
    rc = res[res['ccy']==c].copy()
    weeks_available = len(rc)
    weeks_traded = int(rc['active'].sum())
    active_frac = weeks_traded / weeks_available if weeks_available>0 else np.nan

    r_cond = rc.loc[rc['active']==1,'ret'].dropna()
    r_uncond = rc['ret'].fillna(0.0)

    sw = sharpe_weekly(r_cond) if len(r_cond)>1 else np.nan
    # per currency wealth with zero return when inactive
    w,dd = wealth_dd_from_returns(r_uncond)

    stats.append({
        'ccy':c,
        'weeks_available':weeks_available,
        'weeks_traded':weeks_traded,
        'active_frac':active_frac,
        'mean_weekly_ret_cond_active':float(r_cond.mean()) if len(r_cond)>0 else np.nan,
        'vol_weekly_ret_cond_active':float(r_cond.std(ddof=1)) if len(r_cond)>1 else np.nan,
        'mean_weekly_ret_uncond':float(r_uncond.mean()),
        'vol_weekly_ret_uncond':float(r_uncond.std(ddof=1)),
        'sharpe_weekly_cond_active':sw,
        'sharpe_ann_cond_active':(np.sqrt(52)*sw if np.isfinite(sw) else np.nan),
        'pnl_sum_usd':float(rc['pnl_usd'].sum()),
        'max_dd_wealth':float(dd.min())
    })
stats_df=pd.DataFrame(stats).sort_values('sharpe_ann_cond_active', ascending=False)
stats_df.to_csv(OUTPUT_DIR/'currency_stats.csv', index=False)

# active diagnostics + spread distribution
a_rows=[]
for c in EM:
    rc=res[res['ccy']==c]
    sp=rc['spread'].dropna()
    a_rows.append({
        'ccy':c,
        'weeks_traded':int(rc['active'].sum()),
        'active_frac':float(rc['active'].mean()),
        'spread_mean':float(sp.mean()),
        'spread_p5':float(sp.quantile(0.05)),
        'spread_p50':float(sp.quantile(0.50)),
        'spread_p95':float(sp.quantile(0.95))
    })
a_rows.append({'ccy':'PORTFOLIO','weeks_traded':int(port['active_positions'].sum()),'active_frac':np.nan,
               'spread_mean':np.nan,'spread_p5':np.nan,'spread_p50':avg_active,'spread_p95':np.nan})
active_diag=pd.DataFrame(a_rows)
active_diag.to_csv(OUTPUT_DIR/'active_diagnostics.csv', index=False)

# currency correlations
ret_zero = res.pivot(index='date', columns='ccy', values='ret').sort_index().fillna(0.0)
ret_active = res.pivot(index='date', columns='ccy', values='ret').sort_index()
ret_zero.corr().to_csv(OUTPUT_DIR/'currency_corr.csv')
ret_active.corr(min_periods=10).to_csv(OUTPUT_DIR/'currency_corr_active_overlap.csv')

# drop-one diagnostics
full_w = sharpe_weekly(port['port_ret'])
full_a = np.sqrt(52)*full_w if np.isfinite(full_w) else np.nan
full_dd = float(port['drawdown'].min())

drop_rows=[{'portfolio':'full','sharpe_weekly':full_w,'sharpe_ann':full_a,'max_dd_wealth':full_dd}]
for c in EM:
    tmp = res[res['ccy']!=c].pivot(index='date', columns='ccy', values='ret').sort_index()
    pr = tmp.mean(axis=1, skipna=True).fillna(0.0)
    w,dd = wealth_dd_from_returns(pr)
    sh = sharpe_weekly(pr)
    drop_rows.append({'portfolio':f'ex_{c}','sharpe_weekly':sh,'sharpe_ann':(np.sqrt(52)*sh if np.isfinite(sh) else np.nan),'max_dd_wealth':float(dd.min())})

drop_df=pd.DataFrame(drop_rows)
drop_df.to_csv(OUTPUT_DIR/'drop_one_diagnostic.csv', index=False)

print('tables written')

In [ ]:
# Market factors (real if available, else proxies) + HAC(4) regressions
factor_hits=[]
for pat in ['*VIX*','*DXY*','*SPX*','*MSCI*','*UST*','*rates*','*factor*']:
    factor_hits += list((BASE/'data_clean').glob(pat))
factor_hits = sorted(set(factor_hits), key=lambda p:p.name)
print('factor files found:', [f.name for f in factor_hits])

# proxy factors
em_fx_basket = np.log(fx_w[EM]).diff().mean(axis=1)
usd_proxy = -np.log(fx_w['GBP']).diff()  # USD strength proxy via GBPUSD inverse
rates_proxy_on = ois_w.diff()
rates_proxy_5y = gbp5_w.diff()

factors = pd.DataFrame({
    'usd_proxy': usd_proxy,
    'em_fx_basket': em_fx_basket,
    'rates_proxy_on': rates_proxy_on,
    'rates_proxy_5y': rates_proxy_5y,
}, index=port.index)

mf = pd.concat([port['port_ret'], factors], axis=1).dropna()
mcorr = mf.corr().loc[factors.columns, ['port_ret']].rename(columns={'port_ret':'corr_with_port'})
mcorr.to_csv(OUTPUT_DIR/'market_factor_corr.csv')


def ols_hac(y, X, lags=4):
    # X includes intercept column
    y=np.asarray(y).reshape(-1,1)
    X=np.asarray(X)
    n,k=X.shape
    beta=np.linalg.inv(X.T@X)@(X.T@y)
    u=(y-X@beta)
    # Newey-West
    S=np.zeros((k,k))
    for t in range(n):
        xt=X[t:t+1].T
        S += float(u[t,0]**2) * (xt@xt.T)
    for L in range(1,lags+1):
        w=1 - L/(lags+1)
        for t in range(L,n):
            xt=X[t:t+1].T; xlag=X[t-L:t-L+1].T
            S += w*float(u[t,0]*u[t-L,0])*(xt@xlag.T + xlag@xt.T)
    XXi=np.linalg.inv(X.T@X)
    V=XXi@S@XXi
    se=np.sqrt(np.diag(V)).reshape(-1,1)
    tstat=(beta/se).flatten()
    yhat=X@beta
    e=y-yhat
    r2=1 - float(((e.T@e)/((y-y.mean()).T@(y-y.mean()))).item())
    return beta.flatten(), tstat, r2, n

reg_rows=[]
y=mf['port_ret'].values
for fac in factors.columns:
    X=np.column_stack([np.ones(len(mf)), mf[fac].values])
    b,t,r2,n = ols_hac(y,X,lags=4)
    reg_rows.append({'model':'univariate','factor':fac,'alpha':b[0],'beta':b[1],'t_beta_hac4':t[1],'r2':r2,'n':n})

multi_cols=['usd_proxy','em_fx_basket','rates_proxy_5y']
X=np.column_stack([np.ones(len(mf))] + [mf[c].values for c in multi_cols])
b,t,r2,n = ols_hac(y,X,lags=4)
reg_rows.append({'model':'multivariate','factor':'const','alpha':b[0],'beta':b[0],'t_beta_hac4':t[0],'r2':r2,'n':n})
for j,c in enumerate(multi_cols, start=1):
    reg_rows.append({'model':'multivariate','factor':c,'alpha':b[0],'beta':b[j],'t_beta_hac4':t[j],'r2':r2,'n':n})

reg_df=pd.DataFrame(reg_rows)
reg_df.to_csv(OUTPUT_DIR/'market_factor_regs.csv', index=False)

# factor figures
for fac in factors.columns:
    tmp=mf[['port_ret',fac]].dropna()
    plt.figure(figsize=(5,4))
    plt.scatter(tmp[fac], tmp['port_ret'], s=10, alpha=0.6)
    z=np.polyfit(tmp[fac], tmp['port_ret'], 1)
    xs=np.linspace(tmp[fac].min(), tmp[fac].max(), 100)
    plt.plot(xs, z[0]*xs+z[1], color='red')
    plt.xlabel(fac); plt.ylabel('port_ret'); plt.tight_layout()
    plt.savefig(FIG_DIR/f'factor_scatter_{fac}.png', dpi=120)
    plt.close()

print(mcorr)
print(reg_df)

In [ ]:
# Figures + report + manifest + verification

# figures
plt.figure(figsize=(9,4)); plt.plot(port.index,port['wealth']); plt.title('Portfolio Wealth'); plt.tight_layout(); plt.savefig(FIG_DIR/'portfolio_wealth.png', dpi=130); plt.close()
plt.figure(figsize=(9,4)); plt.plot(port.index,port['drawdown']); plt.title('Portfolio Drawdown'); plt.tight_layout(); plt.savefig(FIG_DIR/'portfolio_drawdown.png', dpi=130); plt.close()
plt.figure(figsize=(9,4)); plt.plot(port.index,port['active_positions']); plt.title('Active Positions'); plt.tight_layout(); plt.savefig(FIG_DIR/'active_positions.png', dpi=130); plt.close()

corr=pd.read_csv(OUTPUT_DIR/'currency_corr.csv', index_col=0)
plt.figure(figsize=(6,5))
mat=corr.values
plt.imshow(mat,cmap='RdBu_r',vmin=-1,vmax=1); plt.colorbar()
plt.xticks(range(len(corr.columns)),corr.columns,rotation=45,ha='right')
plt.yticks(range(len(corr.index)),corr.index)
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if np.isfinite(mat[i,j]):
            plt.text(j,i,f'{mat[i,j]:.2f}',ha='center',va='center',fontsize=8)
plt.tight_layout(); plt.savefig(FIG_DIR/'corr_heatmap.png', dpi=130); plt.close()

# report generation from outputs
stats=pd.read_csv(OUTPUT_DIR/'currency_stats.csv')
drop=pd.read_csv(OUTPUT_DIR/'drop_one_diagnostic.csv')
port_out=pd.read_csv(OUTPUT_DIR/'portfolio_weekly_returns.csv', parse_dates=['date'])
mc=pd.read_csv(OUTPUT_DIR/'market_factor_corr.csv', index_col=0)
mr=pd.read_csv(OUTPUT_DIR/'market_factor_regs.csv')
miss=pd.read_csv(OUTPUT_DIR/'alignment_missingness.csv')
active=pd.read_csv(OUTPUT_DIR/'active_diagnostics.csv')


def md_table(df, nd=6):
    cols=list(df.columns)
    lines=['| '+' | '.join(cols)+' |','|'+'|'.join(['---']*len(cols))+'|']
    for _,r in df.iterrows():
        vals=[]
        for c in cols:
            v=r[c]
            if isinstance(v,float): vals.append(f'{v:.{nd}f}')
            else: vals.append(str(v))
        lines.append('| '+' | '.join(vals)+' |')
    return '\n'.join(lines)

text=[]
text.append('# HW6 FX Carry Report')
text.append('## Spec Recap')
text.append('- Weekly USD 10MM lending, GBP 8MM borrowing at OIS+50bp, entry filter uses GBP 5Y funding rate: s5_lend >= s5_fund + 50bp.')
text.append('- Lending MTM shifts all coupon/principal times by 1/52 and reprices on exit curve (dirty pricing via shifted cashflows).')
text.append('## Data & Coverage')
text.append('- EM curves sourced from ../../Data/ with fallback ./data (manifest has exact files).')
text.append('- OIS/FX/funding curve raw from ./data_clean/.')
text.append(f"- Weekly window: {port_out['date'].min().date()} to {port_out['date'].max().date()}, N={len(port_out)}.")
text.append('## Funding 5Y Construction')
text.append('- GBP 5Y funding series constructed from local BoE raw OIS spot-curve file (not overnight proxy).')
text.append('- Overnight OIS is used only for borrowing accrual +50bp.')
text.append('## Methodology')
text.append('- Par-curve bootstrap and discounting follow Zero/Spot curve conventions (linear interpolation + recursive DF solve).')
text.append('- Cashflow schedule shift validation: times_exit_full = times_entry - 1/52 with assertion in notebook.')
text.append('## Results')
text.append(md_table(stats))
text.append(md_table(port_out[['port_ret','wealth','drawdown','active_positions']].describe().T))
text.append('![wealth](outputs/figures/portfolio_wealth.png)')
text.append('![drawdown](outputs/figures/portfolio_drawdown.png)')
text.append('![active](outputs/figures/active_positions.png)')
text.append('![corr](outputs/figures/corr_heatmap.png)')
text.append('## Drop-One Diagnostics')
text.append(md_table(drop))
text.append('## Market Factors')
text.append(md_table(mc.reset_index()))
text.append(md_table(mr))
text.append('Carry framing: factor signs are consistent with carry-vs-crash intuition where USD/risk shocks can dominate carry accrual.')
text.append('## Robustness & Guardrails')
text.append(md_table(miss))
text.append(md_table(active))

report_path = BASE/'hw6_fx_carry_report.md'
report_path.write_text('\n\n'.join(text))

# finalize manifest
manifest['coverage']['start'] = str(port_out['date'].min().date())
manifest['coverage']['end'] = str(port_out['date'].max().date())
manifest['coverage']['weekly_obs'] = int(len(port_out))
manifest['coverage']['currencies'] = EM
(OUTPUT_DIR/'run_manifest.json').write_text(json.dumps(manifest, indent=2))

# verification
required_csv = {
 'currency_stats.csv': ['ccy','weeks_available','weeks_traded','active_frac','sharpe_weekly_cond_active','sharpe_ann_cond_active'],
 'currency_corr.csv': [],
 'drop_one_diagnostic.csv': ['portfolio','sharpe_weekly','sharpe_ann','max_dd_wealth'],
 'portfolio_weekly_returns.csv': ['date','port_ret','wealth','drawdown','active_positions'],
 'market_factor_corr.csv': ['corr_with_port'],
 'market_factor_regs.csv': ['model','factor','beta','t_beta_hac4','r2','n'],
 'active_diagnostics.csv': ['ccy','weeks_traded','active_frac']
}
for fn, cols in required_csv.items():
    fp=OUTPUT_DIR/fn
    if (not fp.exists()) or fp.stat().st_size==0:
        raise RuntimeError(f'Missing/empty output: {fn}')
    df=pd.read_csv(fp)
    for c in cols:
        if c not in df.columns:
            raise RuntimeError(f'Column {c} missing in {fn}')

for fig in ['portfolio_wealth.png','portfolio_drawdown.png','corr_heatmap.png','active_positions.png']:
    fp=FIG_DIR/fig
    if (not fp.exists()) or fp.stat().st_size==0:
        raise RuntimeError(f'Missing/empty figure: {fig}')

if (not report_path.exists()) or report_path.stat().st_size==0:
    raise RuntimeError('Report missing/empty')
rt=report_path.read_text()
for sec in ['Market Factors','Funding 5Y Construction']:
    if sec not in rt:
        raise RuntimeError(f'Report section missing: {sec}')

ad = pd.read_csv(OUTPUT_DIR/'active_diagnostics.csv')
avg_diag = float(ad.loc[ad['ccy']=='PORTFOLIO','spread_p50'].iloc[0])
avg_port = float(pd.read_csv(OUTPUT_DIR/'portfolio_weekly_returns.csv')['active_positions'].mean())
if abs(avg_diag-avg_port) > 1e-6:
    raise RuntimeError('Active diagnostics mismatch with portfolio active_positions mean')

print('Verification PASSED')